In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
#pip install torch

In [3]:
import torch
import numpy as np

In [4]:
from importlib import reload
import gedi_adapter
reload(gedi_adapter)
from gedi_adapter import GediAdapter

In [5]:
'''import modeling_utils
import text_processing
import modeling_gpt2
import gedi_training'''

'import modeling_utils\nimport text_processing\nimport modeling_gpt2\nimport gedi_training'

In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\shuba\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [7]:

from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
t5name = 'SkolkovoInstitute/t5-paraphrase-paws-msrp-opinosis-paranmt'

In [8]:
import sys
sys.path.append(os.path.abspath('../transfer_utils/'))

import text_processing
reload(text_processing);

In [9]:
tokenizer = AutoTokenizer.from_pretrained(t5name)

In [10]:
para = AutoModelForSeq2SeqLM.from_pretrained(t5name)
para.resize_token_embeddings(len(tokenizer)) 

Embedding(32100, 768)

In [11]:
model_path = 'SkolkovoInstitute/gpt2-base-gedi-detoxification'
gedi_dis = AutoModelForCausalLM.from_pretrained(model_path)

Some weights of the model checkpoint at SkolkovoInstitute/gpt2-base-gedi-detoxification were not used when initializing GPT2LMHeadModel: ['bias', 'logit_scale']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
NEW_POS = tokenizer.encode('normal', add_special_tokens=False)[0]
NEW_NEG = tokenizer.encode('toxic', add_special_tokens=False)[0]

In [13]:
# add gedi-specific parameters
if os.path.exists(model_path):
    w = torch.load(model_path + '/pytorch_model.bin', map_location='cpu')
    gedi_dis.bias = w['bias']
    gedi_dis.logit_scale = w['logit_scale']
    del w
else:
    gedi_dis.bias = torch.tensor([[ 0.08441592, -0.08441573]])
    gedi_dis.logit_scale = torch.tensor([[1.2701858]])
print(gedi_dis.bias, gedi_dis.logit_scale)

tensor([[ 0.0844, -0.0844]]) tensor([[1.2702]])


In [14]:
%%time
from gedi_adapter import GediAdapter
import transformers
#import modeling_utils
dadapter = GediAdapter(model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=5, target=1, neg_code=NEW_NEG, pos_code=NEW_POS, lb=None, ub=None)
text = 'The internal policy of Trump is flawed.'
print('====BEFORE====')
print(text)
print('====AFTER=====')
#input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0).to(para.device)
inputs = tokenizer.encode(text, return_tensors='pt').to(para.device)
print("hey")
result=dadapter.generate(inputs, num_return_sequences=3, do_sample=True, temperature=1.0, repetition_penalty=3.0,num_beams=1)
for r in result:
    print(tokenizer.decode(r, skip_special_tokens=True))

====BEFORE====
The internal policy of Trump is flawed.
====AFTER=====
hey


C:\Users\shuba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


the president's a corruption to Iran and Iraq is corrupt.
the trumps is fooling themselves with his policies!
the president's head is wrong in his own personal sphere; not to mention, his
CPU times: total: 16.2 s
Wall time: 4.09 s


In [15]:
import torch
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [16]:
import gc

def cleanup():
    gc.collect()
    if torch.cuda.is_available() and device.type != 'cpu':
        with torch.cuda.device(device):
            torch.cuda.empty_cache()

In [17]:
para.to(device);
para.eval();


In [18]:
gedi_dis.to(device);
gedi_dis.bias = gedi_dis.bias.to(device)
gedi_dis.logit_scale = gedi_dis.logit_scale.to(device)
gedi_dis.eval();

In [19]:
with open('../../data/test/test_10k_toxic', 'r') as f:
    test_data = [line.strip() for line in f.readlines()]
print(len(test_data))

10000


In [20]:
dadapter = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=10, target=0, neg_code=NEW_NEG, pos_code=NEW_POS, 
    reg_alpha=3e-5, ub=0.01
)

In [21]:
def paraphrase(text, n=None, max_length='auto', beams=2):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(dadapter.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    result = dadapter.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, temperature=0.0, repetition_penalty=3.0, max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    texts = [text_processing.text_postprocess(t) for t in texts]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [22]:
paraphrase(test_data[:3])

["You'd be a bad guy. I mean, you'd have to do",
 "'Lose One', as good and bad as the boss.",
 'A bad society does bad things, and votes for bad politicians.']

# Evaluate

In [23]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf_name = 'SkolkovoInstitute/roberta_toxicity_classifier_v1'
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
import numpy

In [25]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [26]:
import torch
predict_toxicity(['hello world', 'hello aussie', 'hello fucking bitch'])

array([5.0525083e-05, 8.7885252e-05, 9.9968088e-01], dtype=float32)

# The baseline

In [44]:
# reload(gedi_adapter)
from gedi_adapter import GediAdapter


adapter2 = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, 
    gedi_logit_coef=10, 
    target=0, pos_code=NEW_POS, 
    neg_code=NEW_NEG,
    reg_alpha=3e-5,
    ub=0.01,
    untouchable_tokens=[0, 1],
)


def paraphrase(text, max_length='auto', beams=5, rerank=True):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(adapter2.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    attempts = beams
    out = adapter2.generate(
        inputs, 
        num_beams=beams,
        num_return_sequences=attempts, 
        do_sample=False, 
        temperature=1.0, 
        repetition_penalty=3.0, 
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        output_scores=True, 
        return_dict_in_generate=True,
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]

    if rerank:
        scores = predict_toxicity(results)
    
    results = [text_processing.text_postprocess(t) for t in results]
    out_texts = []
    for i in range(len(texts)):
        if rerank:
            idx = scores[(i*attempts):((i+1)*attempts)].argmin()
        else:
            idx = 0
        out_texts.append(results[i*attempts+idx])
    return out_texts

torch.manual_seed(0)
paraphrase(['this whole article is bullshit!', 'bitch I warned yall. didnt i?', 'can someone please fix that crap', 'Its a very stupid name'], beams=3)

["This article's bad.",
 "Bits I've warned you.",
 'I was an ad.',
 "it'll be a very strange name."]

In [42]:
paraphrase(['this whole article is bullshit!', 'bitch I warned yall. didnt i?', 'can someone please fix that crap', 'Its a very stupid name'], beams=3, rerank=False)

['This article is a mess!',
 'Bite I warned you.',
 "I'd like someone to fix that.",
 "it's a very bad name."]

In [49]:
paraphrase(['I was such an asshole', 'bitch I warned yall. didnt i?', 'can someone please fix that crap', 'Its a very stupid name'], beams=25, rerank=False)

['I was an apex.',
 'Bite I warned you.',
 "I'd like someone to fix that.",
 "it's a very bad name."]

In [30]:
paraphrase(['I was such ana!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=10, rerank=True)

["Fick 'Emmy.",
 "I'd like to thank you for that!",
 "it'll remind me of a duck.",
 'He has to be a TSA.']

In [31]:
batch_size = 2

In [32]:
import os
from tqdm.auto import tqdm, trange

cleanup()

lines = test_data[:10]


for i in trange(int(len(lines) / batch_size + 1)):
    if i % 10 == 0:
        cleanup()
    t = i * batch_size
    batch = [line.strip() for line in lines[t:(t+batch_size)]]
    if not batch:
        continue

    try:
        res = paraphrase(batch, max_length='auto', beams=10)
    except RuntimeError:
        print('runtime error for batch ', i)
        try:
            cleanup()
            res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]
        except RuntimeError:
            print('runtime error for batch ', i, 'even with batch size 1')
            res = batch
            cleanup()
    for out in res:
        print(out)

  0%|          | 0/6 [00:00<?, ?it/s]

You'd be right. You'd be right. You'd
a boss who's at the top of his game.
A bad society does the wrong things, and votes for the wrong politicians.
a president who's not.
Colberg's a bad guy, and I'm a TSA.
"Studden,studden,studden"
I'd like to thank you, I think it's an error and the contributions are unwise.
Luna's oh-so-stunnel, immature girl......who has no idea how to do it, which smells bad, I'd like to see
Mormonis is the wrong thing to do, it's the wrong thing to do, it's the wrong thing to do, it's the wrong thing to do, it's the right thing to do, it's the right thing to do, it's the
First of all, it'll be here. - It'll be here. - It'll be here. - It'll be here. - It'll be here. - It'll be there. - It'll be here.


Expected output:

```
You'd be right. You'd be right. You'd
As snooty and overbearing as its
A bad society does the wrong things, and votes for the wrong politicians.
President, he's the man. He's the man.
Colberg's a bad guy, and I'm a TSA.
Dubious, dubious, dubious, dubious.
I like you. I think you'll be an intelligent man and your contributions will be a big deal.
Luna's oh-so-stunnel, immature girl......who has no idea how to do it, which smells bad, I'd like to see
Mormonis is the wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The right thing to do. The right thing to do. The right thing to do 
You'd be a bad guy, uninitiated.
```